In [2]:
class WhileLoop:
    def __init__(self, condition, content, code_before_while, all_code_after_while):
        self.condition = condition
        self.content = content
        self.code_before_while = code_before_while
        self.all_code_after_while = all_code_after_while

In [3]:
import re

def extract_single_outer_while(source):
    indent_regex = re.compile(r'^\s+', flags=re.MULTILINE)
    try: 
        code_before_while = re.sub(indent_regex, '', source[:source.index('while')].strip())
    except ValueError:
        return WhileLoop(None, None, source, None)

    condition_start = re.search('while\s*\(', source).end()
    condition_end = source.index(')', condition_start)
    condition = source[condition_start:condition_end].strip()
    
    # search for the end } by matching the number of { and } in the content
    content_start = source.index('{', condition_end) + 1
    brace_count = 1
    for i, c in enumerate(source[content_start:]):
        if c == '{':
            brace_count += 1
        elif c == '}':
            brace_count -= 1
            if brace_count == 0:
                content_end = content_start + i
                break

    content = re.sub(indent_regex, '', source[content_start:content_end].strip())
    all_code_after_while = re.sub(indent_regex, '', source[content_end+1:].strip())
    return WhileLoop(condition, content, code_before_while, all_code_after_while)

In [4]:
# test for extract_single_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loop = extract_single_outer_while(source)
print("================================")
print(loop.condition)
print("================================")
print(loop.content)
print("================================")
print(loop.code_before_while)
print("================================")
print(loop.all_code_after_while)

y <= y2
int x = x1;
while (x <= xR)
{
plot = 1; 
x = x + 1;
}
while (errR < 0)
{
xR = xR + 1;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y = y + 1;
int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
int counter = 0;
while (counter < 10)
{
counter++;
}
counter *= 2;


In [5]:
def extract_multiple_outer_while(source):
    # (node, depth)
    all_loops = []
    all_loops.append((extract_single_outer_while(source), 0))
    while all_loops[-1][0].all_code_after_while and all_loops[-1][0].all_code_after_while.strip():
        all_loops.append((extract_single_outer_while(all_loops[-1][0].all_code_after_while), all_loops[-1][1]+1))
    return all_loops

In [6]:
# test for extract_multiple_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loops = extract_multiple_outer_while(source)
for loop in loops:
    print(loop[0].code_before_while)
    if (loop[0].condition): 
        print("while (", loop[0].condition, ") {")
        print(loop[0].content)
        print("}")


int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
while ( y <= y2 ) {
int x = x1;
while (x <= xR)
{
plot = 1; 
x = x + 1;
}
while (errR < 0)
{
xR = xR + 1;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y = y + 1;
}
int counter = 0;
while ( counter < 10 ) {
counter++;
}
counter *= 2;


In [7]:
class WNode:
    def __init__(self, data, children, condition=False):
        self.data = data
        self.children = children
        self.condition = condition
        self.variables = {}

def build_tree(content):
    if (extract_single_outer_while(content).condition == None):
        return WNode(content, [])
    all_loops = []
    all_loops.append(extract_single_outer_while(content))
    while all_loops[-1].all_code_after_while and all_loops[-1].all_code_after_while.strip():
        all_loops.append(extract_single_outer_while(all_loops[-1].all_code_after_while))
    children = []
    for loop in all_loops:
        if loop.content: 
            children.append(WNode(loop.code_before_while, []))
            children.append(WNode(loop.condition, [build_tree(loop.content)], True))
        else: 
            children.append(WNode(loop.code_before_while, []))
    return WNode('', children)

def print_while_tree(node, indent=-1, childNum=0):
    if node.condition:
        print('  '*indent + 'while (' + node.data + ') {', '-> child:', childNum)
    else: 
        for line in node.data.splitlines():
            print('  '*indent + line, '-> child:', childNum)
    for idx, child in enumerate(node.children):
        print_while_tree(child, indent+1, idx) 
    if node.variables: 
        print('  '*indent + '-> Variables: ')
        for key, value in node.variables.items():
            print('  '*(indent+1) + '-> ' + key + ': ', end='')
            for mutation in value:
                print(mutation, end=', ')
            print()
    if node.condition:
        print('  '*indent + '}')

In [8]:
# test for build_tree(source)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)

print_while_tree(tree)

int errR = (y2 - y1) - 2 * (x2 - x1); -> child: 0
int xR = x1; -> child: 0
int y = y1; -> child: 0
while (y <= y2) { -> child: 1
    int x = x1; -> child: 0
    while (x <= xR) { -> child: 1
      plot = 1;  -> child: 0
      x = x + 1; -> child: 0
    }
    while (errR < 0) { -> child: 3
      xR = xR + 1; -> child: 0
      errR = errR + 2 * (y2 - y1); -> child: 0
    }
    errR = errR - 2 * (x2 - x1); -> child: 4
    y = y + 1; -> child: 4
}
int counter = 0; -> child: 2
while (counter < 10) { -> child: 3
  counter++; -> child: 0
}
counter *= 2; -> child: 4


In [9]:
import re

def extract_variables(code):
    # Extract variables that start with a letter
    variables = re.findall(r'\b[a-zA-Z]\w*\b', code)
    # remove keywords in C, such as int, float, double, etc.
    variables = [var for var in variables if var not in ['int', 'float', 'double', 'char', 'long', 'short', 'unsigned', 'signed', 'void', 'struct', 'union', 'enum', 'typedef', 'const', 'volatile', 'auto', 'register', 'static', 'extern', 'inline', 'restrict', 'bool', 'complex', 'imaginary', 'break', 'case', 'continue', 'default', 'do', 'else', 'for', 'goto', 'if', 'return', 'sizeof', 'switch', 'while', 'alignas', 'alignof', 'atomic', 'noreturn', 'static_assert', 'thread_local', 'true', 'false', 'NULL']]
    return variables

def extract_variables_from_tree(node):
    variables = []
    for line in node.data.splitlines():
        variables += extract_variables(line)
    for child in node.children:
        variables += extract_variables_from_tree(child)
    # remove duplicates and sort
    variables = list(dict.fromkeys(variables))
    variables.sort()
    return variables

In [10]:
# test for extract_variables_from_tree(tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)
variables = extract_variables_from_tree(tree)
print(variables)

['counter', 'errR', 'plot', 'x', 'x1', 'x2', 'xR', 'y', 'y1', 'y2']


In [11]:
def extract_modifications(variable, code): 
    # Extract modifications of the variable
    modifications = re.findall(r'\b' + variable + r'\b\s*=\s*[^;]+', code)
    # keep only the right side of the assignment
    modifications = [re.sub(r'\b' + variable + r'\b\s*=\s*', '', modification) for modification in modifications]
    return modifications

In [12]:
# test for extract_modifications(variable, code)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)
variable = 'errR'
node = tree.children[1].children[0].children[3].children[0].data
print(node)
modifications = extract_modifications(variable, node)
print(modifications)

xR = xR + 1;
errR = errR + 2 * (y2 - y1);
['errR + 2 * (y2 - y1)']


In [13]:
class Mutation: 
    def __init__(self, modification, condition):
        self.modification = modification
        self.condition = condition
    
    def __str__(self):
        if self.condition == None or not self.condition.strip():
            return self.modification
        return self.modification + ' if ' + self.condition

def generate_variable_tree(variable, root, modifications): 
    if root == None or root.children == []:
        return None
    for child in root.children: 
        if variable in extract_variables(child.data): 
            if extract_modifications(variable, child.data): 
                modifications.append(Mutation(extract_modifications(variable, child.data)[0], root.data)) # TODO: fix the fact that extract_modifications returns a list
        if variable in extract_variables_from_tree(child):
            generate_variable_tree(variable, child, modifications) 
            local_modifications = [Mutation(modification.modification, modification.condition) for modification in modifications]
            # check if variable is in the first modification using regex
            if local_modifications and local_modifications[0].modification.strip() and re.search(r'\b' + variable + r'\b', local_modifications[0].modification):
                # append the variable itself as the first modification
                local_modifications.insert(0, Mutation(variable, root.data))
            child.variables[variable] = local_modifications
    return root

In [14]:
# test for generate_variable_trees(variables, tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source).children[1]
# print_while_tree(tree)
variable = 'errR'
tree = generate_variable_tree(variable, tree, [])
# print_while_tree(tree)

variable = 'y'
tree = generate_variable_tree(variable, tree, [])
print_while_tree(tree)

while (y <= y2) { -> child: 0
  int x = x1; -> child: 0
  while (x <= xR) { -> child: 1
    plot = 1;  -> child: 0
    x = x + 1; -> child: 0
  }
  while (errR < 0) { -> child: 3
    xR = xR + 1; -> child: 0
    errR = errR + 2 * (y2 - y1); -> child: 0
    -> Variables: 
      -> errR: errR if errR < 0, errR + 2 * (y2 - y1) if errR < 0, 
  -> Variables: 
    -> errR: errR, errR + 2 * (y2 - y1) if errR < 0, 
  }
  errR = errR - 2 * (x2 - x1); -> child: 4
  y = y + 1; -> child: 4
  -> Variables: 
    -> errR: errR, errR + 2 * (y2 - y1) if errR < 0, errR - 2 * (x2 - x1), 
    -> y: y, y + 1, 
-> Variables: 
  -> errR: errR if y <= y2, errR + 2 * (y2 - y1) if errR < 0, errR - 2 * (x2 - x1), 
  -> y: y if y <= y2, y + 1, 
}


In [44]:
class MutationNode(object):
    def __init__(self, condition=None, mutated=None, true_branch=None, false_branch=None):
        self.condition = condition
        self.mutated = mutated
        self.true_branch = true_branch
        self.false_branch = false_branch

def combine_modifications(variable, modifications): 
    # if not modifications: 
    #     return []
    # combined = modifications[0]
    # for modification in modifications[1:]:
    #     # search for the variable in the modification
    #     if re.search(r'\b' + variable + r'\b', modification):
    #         # if the variable is in the modification, replace it with the combined modification
    #         combined = re.sub(r'\b' + variable + r'\b', combined, modification)
    #     else: 
    #         # if the variable is not in the modification, append it to the combined modification
    #         combined += modification
    # return combined
    mutated = ""
    return combine_modifications_helper(variable, mutated, modifications)

def combine_modifications_helper(variable: str, prev_mutated: str, mutations: list[Mutation]) -> MutationNode:
    if not mutations: 
        return None
    mutation = mutations[0]
    node = MutationNode(condition=mutation.condition)
    if re.search(r'\b' + variable + r'\b', prev_mutated):
        # if the variable is in the modification, replace it with the combined modification
        node.mutated = re.sub(r'\b' + variable + r'\b', prev_mutated, mutation.modification)
    else: 
        # if the variable is not in the modification, append it to the combined modification
        node.mutated = prev_mutated + mutation.modification
    if mutations[1:]:
        node.true_branch = combine_modifications_helper(variable, node.mutated, mutations[1:].copy())
        if node.condition.strip():
            node.false_branch = combine_modifications_helper(variable, prev_mutated, mutations[1:].copy())
    return node
    

In [143]:
# test for combine_modifications(variables, tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source).children[1]
variable = 'errR'
tree = generate_variable_tree(variable, tree, [])
modifications = tree.children[0].children[4].variables[variable]
print(modifications)
combined = combine_modifications(variable, modifications)

def print_mutation_tree(node: MutationNode, variable: str, indent: int = 0):
    if node and node.true_branch == None and node.false_branch == None:
        print('  ' * (indent-1) + variable , '<=', node.mutated)
        return   
    if not node: 
        return
    if node.true_branch and node.false_branch and node.true_branch.mutated == node.false_branch.mutated:
        print('  ' * (indent) + variable , '<=', node.true_branch.mutated)
        return
    if node.condition.strip(): 
        print('  ' * (indent-1) + 'if (', node.condition, ') begin')
    indent += 1
    if node.condition.strip(): 
        print_mutation_tree(node.true_branch, variable, indent=indent+1)
    else: 
        print_mutation_tree(node.true_branch, variable, indent=indent)
    if node.condition.strip():
        print('  ' * (indent-2) + 'end else begin')        
        print_mutation_tree(node.false_branch, variable, indent=indent+1)
        print('  ' * (indent-2) + 'end')

print_mutation_tree(combined, variable)

# variable = 'y'
# tree = generate_variable_tree(variable, tree, [])
# modifications = tree.children[0].children[4].variables[variable]
# print(modifications)
# combined = combine_modifications(variable, modifications)
# print(combined)

# variable = 'x'
# tree = generate_variable_tree(variable, tree, [])
# modifications = tree.children[0].children[0].variables[variable]
# print(modifications)
# combined = combine_modifications(variable, modifications)
# print(combined)

# variable = 'y'
# tree = build_tree(source)
# tree = generate_variable_tree(variable, tree, [])
# modifications = tree.children[1].children[0].children[4].variables[variable]
# print(modifications)
# combined = combine_modifications(variable, modifications)
# print(combined)

[<__main__.Mutation object at 0x000002A9A5220EB0>, <__main__.Mutation object at 0x000002A9A5220700>, <__main__.Mutation object at 0x000002A9A5222110>]
if ( errR < 0 ) begin
    errR <= errR + 2 * (y2 - y1) - 2 * (x2 - x1)
end else begin
    errR <= errR - 2 * (x2 - x1)
end


In [144]:
def generate_verilog_from_tree(node, indent=0):
    if node.condition:
        print('  '*indent + 'while (' + node.data + ') begin')
    else: 
        for line in node.data.splitlines():
            # remove keywords in C, such as int, float, double, etc.
            line = re.sub(r'\b(int|float|double|char|long|short|signed|unsigned|void|struct|union|enum|const|volatile|auto|register|static|extern|typedef|sizeof|return|break|continue|if|else|switch|case|default|for|do|while|goto|asm|inline|restrict|_Bool|_Complex|_Imaginary)\b', '', line)
            # match all instances of = in a line and iterate over them
            for match in re.finditer(r'\s*=\s*', line):
                # get the variable name
                variable = line[:match.start()].strip()
                # get the modification
                modification = line[match.end():].strip()
                # check if the variable has been modified in the tree
                if variable in node.variables:
                    # combine the modifications
                    combined = combine_modifications(variable, node.variables[variable])
                    # # replace right hand side of the assignment with the combined modification and replace = with <=
                    # line = re.sub(r'\s*=\s*.*', ' <= ' + combined, line)
                    # combined is a tree structure, with a mutated field and a condition field, print a decision tree for the modifications
                    # print('  '*indent, 'DEBUG 3: ')
                    print_mutation_tree(combined, variable, indent)
                else: 
                    # if the variable has not been modified in the tree, replace the variable with the modification
                    line = re.sub(r'\b' + variable + r'\b', modification, line)
    for idx, child in enumerate(node.children):
        generate_verilog_from_tree(child, indent+1) 
    if node.condition:
        print('  '*indent + 'end')

In [145]:
# use example
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)
# print_while_tree(tree)

tree = tree.children[1]
variables = extract_variables_from_tree(tree)
# print_while_tree(tree)
# print(variables)
for variable in variables:
    tree = generate_variable_tree(variable, tree, [])
# print_while_tree(tree)
generate_verilog_from_tree(tree)

while (y <= y2) begin
  x <= x1
    while (x <= xR) begin
    plot <= 1
      x <= x1x + 1
    end
    while (errR < 0) begin
      xR <= xR + 1
      errR <= errR + 2 * (y2 - y1)
    end
    if ( errR < 0 ) begin
        errR <= errR + 2 * (y2 - y1) - 2 * (x2 - x1)
    end else begin
        errR <= errR - 2 * (x2 - x1)
    end
    y <= y + 1
end


In [146]:
# chatGPT output for generating state machine
def generate_verilog_code():
    # Define the parameters of the state machine
    num_states = 3
    reset_state = 0
    output_state = 2

    # Generate the Verilog code for the state machine
    code = []
    code.append("module state_machine (")
    code.append("  input wire rst,")
    code.append("  input wire clk,")
    code.append("  input wire start,")
    code.append("  output wire done")
    code.append(");")
    code.append("  reg [{}:0] state;".format(num_states-1))
    code.append("  always @(posedge clk) begin")
    code.append("    if (rst) begin")
    code.append("      state <= {};".format(reset_state))
    code.append("    end else begin")
    code.append("      case (state)")
    for i in range(num_states):
        code.append("        {'{:0{}b}'.format(i, num_states)}: begin")
        if i == reset_state:
            code.append("          if (start) begin")
            code.append("            state <= {};".format((i+1) % num_states))
            code.append("          end")
        else:
            code.append("          state <= {};".format((i+1) % num_states))
        code.append("        end")
    code.append("      endcase")
    code.append("    end")
    code.append("  end")
    code.append("  assign done = (state == {});".format(output_state))
    code.append("endmodule")

    # Return the generated code as a single string
    return "\n".join(code)

# Write the generated code to a file
with open("state_machine.sv", "w") as f:
    f.write(generate_verilog_code())